<a href="https://colab.research.google.com/github/Abhiraj36/Malaria-Diagnosis-Using-CNNs/blob/main/Malaria_Detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow_datasets as tfds

dataset, dataset_info = tfds.load(
    'malaria', with_info=True, as_supervised=True, shuffle_files=True, split='train'
)


Data Splitting

In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):

  dataset_size=len(dataset)
  train_dataset = dataset.take(int(0.6*dataset_size))


  val_dataset = dataset.skip(int(TRAIN_RATIO*dataset_size))
  val_dataset = val_dataset.take(int(VAL_RATIO*dataset_size))


  test_dataset = dataset.skip(int((VAL_RATIO*dataset_size)+(TRAIN_RATIO*dataset_size)))
  #print(list(test_dataset.as_numpy_iterator()))
  return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

#dataset = tf.data.Dataset.range(10)
train_dataset, val_dataset, test_dataset = splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

#print(list(train_dataset.take(1).as_numpy_iterator()),  list(test_dataset.take(1).as_numpy_iterator()))


DATA VISUALISATION

In [ ]:
for i,(image, label) in enumerate(train_dataset.take(16)):
  ax=plt.subplot(4, 4, i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

In [ ]:
dataset_info.features['label'].int2str(0)

DATA PREPROCESSING

In [ ]:
#Image resizing and then normalisation
IM_SIZE = 224
def resizing_rescale(image,label):
  return tf.image.resize(image,(IM_SIZE, IM_SIZE))/255.0,label
train_dataset = train_dataset.map(resizing_rescale)

In [ ]:
for image, label in train_dataset.take(1):
  print(image, label)

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size = 1000, reshuffle_each_iteration =True).batch(32).prefetch(tf.data.AUTOTUNE)

Model Creation

In [ ]:
lenet_model = tf.keras.Sequential([
                     InputLayer(shape=(IM_SIZE, IM_SIZE, 3)),
                     Conv2D(filters = 6, kernel_size = 5, strides = 1, padding = 'valid', activation = 'sigmoid'),
                     MaxPool2D(pool_size = 2, strides =2),

                     Conv2D(filters = 6, kernel_size = 5, strides = 1, padding = 'valid', activation = 'sigmoid'),
                     MaxPool2D(pool_size = 2, strides =2),

                     Flatten(),

                     Dense(1000, activation = "sigmoid"),
                     Dense(100, activation = "sigmoid"),
                     Dense(1, activation = "sigmoid"),

])

lenet_model.summary()

In [ ]:
y_true = [0,1,0,0]
y_pred = [0.6, 0.5, 0.94, 1]

bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred)


In [ ]:
lenet_model.compile(optimizer=Adam(learning_rate = 0.1),
              loss = BinaryCrossentropy(), )


In [ ]:
history = lenet_model.fit(train_dataset, validation_data = val_dataset, epochs = 100, verbose = 1)